In [81]:
# Import libraries
# Main libraries 
import numpy as np
import pandas as pd

#Outliers
from scipy import stats
from dfcols import all_square_cols

#Feature Engineering
from sklearn.feature_selection import SelectFromModel
# Balancing Libraries
import imblearn
from imblearn.over_sampling import SMOTE

#Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier

#Testing
from sklearn.metrics import f1_score

In [82]:
# Read files to pandas dataframes
train = pd.read_csv('data/Train.csv')
test = pd.read_csv('data/Test.csv')
sample_submission = pd.read_csv('data/SampleSubmission.csv')


In [83]:
#Outlier 
#Mean replacement of outliers
# outlierAttributes = all_square_cols("elevation") + all_square_cols("slope") + all_square_cols("placurv") + all_square_cols("procurv") + all_square_cols("lsfactor")
# Xtr = Xtr.copy(deep=True)
# for column in Xtr:
#     if (column in outlierAttributes):
#         curcol = train[column]
#         mean = curcol[np.abs(stats.zscore(curcol)) < 3].mean()
#         Xtr.loc[np.abs(stats.zscore(curcol)) >= 3, column] = np.nan
#         Xtr.fillna(mean, inplace=True)

In [84]:

#Feature Transformation
# elevationCols = all_square_cols("slope")

# elevationCols = train[elevationCols]

# for col in elevationCols:
#     curCol = elevationCols[col]
#     elevationCols[col] = pd.cut(curCol, [0,5,22,34,55,100], labels=[0,1,2,3,4]) # here

# train_binning = train.replace(elevationCols)
# train_binning.describe(include = 'all')
#Select

In [85]:
#Split train-test
Xtr = train.head(8000)
Xte = train.tail(2863)
test_in = Xte.drop(columns=['Sample_ID','Label'])
test_res = Xte[["Label"]]

In [86]:
#Balacing
oversample = SMOTE()
XtB, ytB = oversample.fit_resample(Xtr.drop(columns=['Sample_ID','Label']), Xtr[['Label']])

In [87]:
#Model Training + prediction
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(150,100,75,50), random_state=1).fit(XtB,ytB)
y_pred = clf.predict(test_in)
f1 = f1_score(y_pred,test_res)
print(f1)

/Applications/anaconda3/envs/StartHack/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5850340136054422


/Applications/anaconda3/envs/StartHack/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [88]:
y_pred = clf.predict(test.drop(columns=['Sample_ID']))
out = test['Sample_ID']
out = pd.DataFrame({'Sample_ID':test.Sample_ID,'Label':y_pred})
#out.to_csv('pred1Le.csv',index=False)

# Test results
Splt Feature-Transformation Outlier-Method Balancing Model Training Score
70:30 None Mean-Method SMOTE RandomForests 0.6535
70:30 None None SMOTE RandomForests 0.6535